# Steps for running on Anaconda

In [1]:
#STEPS FOR RUNNING ON ANACONDA

#These only need to be done once. We suggest running this notebook in a new anaconda environment.
#Steps below are commented out since they only need to be done once. You may need to restart environment after installing.

#!git clone https://github.com/ultralytics/yolov5  # clone repo
#%cd yolov5
#%pip install -qr requirements.txt
#%pip install wandb

# Steps for running on Jetson

# Code

In [8]:
import cv2
import torch
import os
import shutil
import time
import numpy as np
from IPython.display import Image, clear_output  # to display images

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 1.11.0 (NVIDIA GeForce GTX 1080 Ti)


In [12]:
# Model
model = torch.hub.load('ultralytics/yolov5', 'custom', path='weights/best.pt')

Using cache found in C:\Users\thwang99/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2022-6-14 Python-3.8.13 torch-1.11.0 CUDA:0 (NVIDIA GeForce GTX 1080 Ti, 11264MiB)

Fusing layers... 
Model summary: 213 layers, 7037095 parameters, 0 gradients
Adding AutoShape... 


In [13]:
cap = cv2.VideoCapture(0)

timebasedrow = time.time()
timebasedist = time.time()
timerundrow = time.time()
timerundist = time.time()

while cap.isOpened():
    ret, frame = cap.read()

    # Make detections 
    results = model(frame, size=480)
    
    # print result
    info = results.pandas().xyxy[0]
    
    if len(info) == 0: # no detection 
        # reset distracted and drowsy timers
        timebasedrow = time.time()
        timebasedist = time.time()
        timerundrow = time.time()
        timerundist = time.time()
    else: # detected something
        if info['class'].values[0] ==0: 
            # reset distracted and drowsy timers
            timebasedrow = time.time()
            timebasedist = time.time()
            timerundrow = time.time()
            timerundist = time.time()
        elif info['class'].values[0] > 0 and info['class'].values[0] <= 7:
            timerundist = time.time()
            if((timerundist-timebasedist)>2):
                print("Distracted Driver Detected...{}..{} ms".format(info['name'].values[0],timerundist-timebasedist))
                timebasedist = time.time()  # reset distracted timer
                timerundist = time.time()         
        elif info['class'].values[0] > 7:
            timerundrow = time.time()
            if((timerundrow-timebasedrow)>2):
                print("Drowsy Driver Detected...{}..{} ms".format(info['name'].values[0],timerundrow-timebasedrow))
                timebasedrow = time.time() # reset drowsy timer
                timerundrow = time.time()
        
    
    # show class on the captured image
    cv2.imshow('YOLO', np.squeeze(results.render()))
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

Distracted Driver Detected...c7 - Talking to Passenger..2.005002975463867 ms
Distracted Driver Detected...c1 - Texting - right..2.026440143585205 ms
Distracted Driver Detected...c1 - Texting - right..2.0245466232299805 ms
Distracted Driver Detected...c6 - Hair and Makeup..2.0579659938812256 ms
Distracted Driver Detected...c1 - Texting - right..2.0279531478881836 ms
Distracted Driver Detected...c7 - Talking to Passenger..2.0175397396087646 ms
Distracted Driver Detected...c7 - Talking to Passenger..2.0247957706451416 ms
Distracted Driver Detected...c7 - Talking to Passenger..2.0099966526031494 ms
Distracted Driver Detected...c7 - Talking to Passenger..2.239264488220215 ms
Distracted Driver Detected...c7 - Talking to Passenger..2.0336055755615234 ms
Distracted Driver Detected...c7 - Talking to Passenger..2.0332725048065186 ms
Distracted Driver Detected...c7 - Talking to Passenger..2.020890951156616 ms
Distracted Driver Detected...c7 - Talking to Passenger..2.014958620071411 ms
